# SQLAlchemy Practice

## Set-Up

Begin by importing packages.

In [1]:
# SQLAlchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String # datatypes
from sqlalchemy import ForeignKey

from sqlalchemy import and_, or_, asc, desc, between # conjunctions
from sqlalchemy import text # for textual statements

from sqlalchemy.sql import select 
from sqlalchemy.sql import alias # represents 'as' in sql
from sqlalchemy.sql import func # standard sql functions

from sqlalchemy.ext.declarative import declarative_base

from sqlalchemy.orm import relationship # describe relationships between tables
from sqlalchemy.orm import sessionmaker, Query

# Misc
import os

In the course of the notebook some files are created. We delete them if they already exist for a clean slate.

In [2]:
def remove_file(path):
    if os.path.isfile(path):
        os.remove(path)

        
COLLEGE_DB_PATH = 'college.db'
SALES_DB_PATH = 'sales.db'

remove_file(COLLEGE_DB_PATH)
remove_file(SALES_DB_PATH)

**SQLAlchemy** is a Python toolkit for dealing with databases. It has two ways of interacting with them.

**SQLAlchemy Core:** Uses SQL Expression Language that provides schema-centric usage paradigm
- **SQL Expression Language** allows you represent structures in a relational database using Python
- You can specify SQL statements in Python and use it directly
- This is the closest part to raw SQL in SQLAlchemy

**SQLAlchemy ORM:** Allows classes to be mapped to tables in the database to provide domain-centric usage.

## Resources

Quick Guide: https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_quick_guide.htm

## SQLAlchemy Core

### Connecting to Database

**Engine:** Class that provides a source of database connectivity and behaviour.

**MetaData:** A collection of *Table* objects and associated schema constructs
- Has an optional binding to an engine

In [3]:
# Define local sqlite database to be hosted in repo
engine = create_engine(
    os.path.join('sqlite:///', COLLEGE_DB_PATH), # Different syntax for different database types
    echo = True # sets up SQLAlchemy logging - print SQL used
)
meta = MetaData()

# Define table
students = Table(
   'students', meta, 
   Column('id', Integer, primary_key = True), 
   Column('name', String), 
   Column('lastname', String), 
)

# Use engine object to create table and other objects and store info in metadata
# This creates the local database
meta.create_all(engine)

2021-06-10 16:40:36,517 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-10 16:40:36,517 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2021-06-10 16:40:36,518 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-10 16:40:36,521 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("students")
2021-06-10 16:40:36,523 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-10 16:40:36,526 INFO sqlalchemy.engine.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	lastname VARCHAR, 
	PRIMARY KEY (id)
)


2021-06-10 16:40:36,528 INFO sqlalchemy.engine.Engine [no key 0.00196s] ()
2021-06-10 16:40:36,532 INFO sqlalchemy.engine.Engine COMMIT


**SQL expressions:** Constructed by using methods on tables
- Print corresponding SQL code using `str(<exp>)` (doesn't store specific values)
- Specific values are stored in bind parameter which is visible in compiled form - access using `.compile().params`

**Connection:** Represents an active *DBAPI* (Python Database API) connection
- Run expressions by passing to `execute(<exp>)` method on connection
- This also returns `ResultProxy` object that holds information

In [4]:
# Insert expression
ins = students.insert().values(name = 'Ravi', lastname = 'Kapoor')

# Equivalent SQL insert statement (without specific values)
print(str(ins))

# Access specific parameters
print(ins.compile().params)

INSERT INTO students (name, lastname) VALUES (:name, :lastname)
{'name': 'Ravi', 'lastname': 'Kapoor'}


In [5]:
# Create connection to database
conn = engine.connect()
result = conn.execute(ins)

# Check last set of params inserted
print(result.last_inserted_params())

2021-06-10 16:40:36,550 INFO sqlalchemy.engine.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2021-06-10 16:40:36,554 INFO sqlalchemy.engine.Engine [generated in 0.00327s] ('Ravi', 'Kapoor')
2021-06-10 16:40:36,557 INFO sqlalchemy.engine.Engine COMMIT
{'name': 'Ravi', 'lastname': 'Kapoor'}


In [6]:
# Can insert many rows at once by passing list of dictionaries
conn.execute(students.insert(), [
   {'name':'Rajiv', 'lastname' : 'Khanna'},
   {'name':'Komal','lastname' : 'Bhandari'},
   {'name':'Abdul','lastname' : 'Sattar'},
   {'name':'Priya','lastname' : 'Rajhans'},
]);

2021-06-10 16:40:36,572 INFO sqlalchemy.engine.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2021-06-10 16:40:36,574 INFO sqlalchemy.engine.Engine [generated in 0.00235s] (('Rajiv', 'Khanna'), ('Komal', 'Bhandari'), ('Abdul', 'Sattar'), ('Priya', 'Rajhans'))
2021-06-10 16:40:36,577 INFO sqlalchemy.engine.Engine COMMIT


### Basic Select Statements

**Select:** Either a method on table or function with table as argument
- Limit columns by passing specific columns as argument of `select()` function
- Columns referenced using `<table>.c.<column>` (`c` is an alias for 'column')
- Rename columns using `.label()` method on columns in `select`

**ResultProxy:** Object returned by executing select
- Is iterable with items corresponding to rows
- Has other methods for accessing results: `fetchone()`, `fetchall()`,...

In [7]:
# Select all
s = students.select()
print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

# Run expression
result = conn.execute(s)
print('')

for row in result:
    print(row)

Equivalent SQL
--------------
SELECT students.id, students.name, students.lastname 
FROM students 

2021-06-10 16:40:36,592 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students
2021-06-10 16:40:36,600 INFO sqlalchemy.engine.Engine [generated in 0.00794s] ()

(1, 'Ravi', 'Kapoor')
(2, 'Rajiv', 'Khanna')
(3, 'Komal', 'Bhandari')
(4, 'Abdul', 'Sattar')
(5, 'Priya', 'Rajhans')


In [8]:
# Alternative syntax
s = select(students)
print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

# Run expression
result = conn.execute(s)
result.fetchall()

Equivalent SQL
--------------
SELECT students.id, students.name, students.lastname 
FROM students 

2021-06-10 16:40:36,623 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students
2021-06-10 16:40:36,627 INFO sqlalchemy.engine.Engine [cached since 0.03448s ago] ()


[(1, 'Ravi', 'Kapoor'),
 (2, 'Rajiv', 'Khanna'),
 (3, 'Komal', 'Bhandari'),
 (4, 'Abdul', 'Sattar'),
 (5, 'Priya', 'Rajhans')]

In [9]:
# Specific columns
s = select(
    students.c.name, 
    students.c.lastname.label('surname')
)
print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

# Run expression
result = conn.execute(s)
result.fetchall()

Equivalent SQL
--------------
SELECT students.name, students.lastname AS surname 
FROM students 

2021-06-10 16:40:36,657 INFO sqlalchemy.engine.Engine SELECT students.name, students.lastname AS surname 
FROM students
2021-06-10 16:40:36,659 INFO sqlalchemy.engine.Engine [generated in 0.00121s] ()


[('Ravi', 'Kapoor'),
 ('Rajiv', 'Khanna'),
 ('Komal', 'Bhandari'),
 ('Abdul', 'Sattar'),
 ('Priya', 'Rajhans')]

### Where Clauses

**Where:** Use `.where()` method on `select` and pass column conditions
- Use `where(_and(<cond1>, <cond2>))` for multiple conditions
- Use `where(between(<col>, <val1>, <val2>)` for between

In [10]:
# Select first 3 rows
s = students.select().where(students.c.id <= 3)
print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

print('Params: ', s.compile().params, '\n')

result = conn.execute(s)
print('')
for row in result:
    print(row)

Equivalent SQL
--------------
SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id <= :id_1 

Params:  {'id_1': 3} 

2021-06-10 16:40:36,679 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id <= ?
2021-06-10 16:40:36,682 INFO sqlalchemy.engine.Engine [generated in 0.00363s] (3,)

(1, 'Ravi', 'Kapoor')
(2, 'Rajiv', 'Khanna')
(3, 'Komal', 'Bhandari')


In [11]:
# Select Ravi in first 3 rows
s = students.select().where(
    and_(
        students.c.id <= 3,
        students.c.name == 'Ravi'
    )
)
print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

print('Params: ', s.compile().params, '\n')

result = conn.execute(s)
print('')
for row in result:
    print(row)

Equivalent SQL
--------------
SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id <= :id_1 AND students.name = :name_1 

Params:  {'id_1': 3, 'name_1': 'Ravi'} 

2021-06-10 16:40:36,710 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id <= ? AND students.name = ?
2021-06-10 16:40:36,711 INFO sqlalchemy.engine.Engine [generated in 0.00102s] (3, 'Ravi')

(1, 'Ravi', 'Kapoor')


In [12]:
# Select students with 2nd and 3rd ids
s = students.select().where(
    between(
        students.c.id,
        2,
        3
    )
)

print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

print('Params: ', s.compile().params, '\n')

result = conn.execute(s)
print('')
for row in result:
    print(row)

Equivalent SQL
--------------
SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id BETWEEN :id_1 AND :id_2 

Params:  {'id_1': 2, 'id_2': 3} 

2021-06-10 16:40:36,743 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id BETWEEN ? AND ?
2021-06-10 16:40:36,744 INFO sqlalchemy.engine.Engine [generated in 0.00087s] (2, 3)

(2, 'Rajiv', 'Khanna')
(3, 'Komal', 'Bhandari')


### Textual SQL

**TextClause:** Represents an SQL statement directly
- Constructed by passing SQL query into `text(<string>)`
- Parameters are denoted by colons and passed as kwargs into `execute()`
- Good when SQL is known and static

In [13]:
s = text('SELECT * FROM students WHERE students.id <= :x AND students.name = :y')
result = conn.execute(s, x=3, y='Ravi')
print('')

for row in result:
    print(row)

2021-06-10 16:40:36,758 INFO sqlalchemy.engine.Engine SELECT * FROM students WHERE students.id <= ? AND students.name = ?
2021-06-10 16:40:36,759 INFO sqlalchemy.engine.Engine [generated in 0.00129s] (3, 'Ravi')

(1, 'Ravi', 'Kapoor')


### Some Common Operations

**Alias:** Represents 'AS' in SQL statements
- There is a difference between the name of the variable representing alias table and the alias

**Order By:** Use `order_by(asc(<col>))` or `order_by(desc(<col>))` methods on select

**Functions:** Standard SQL functions are accessed through `func`
- Ex: `func.now()`, `func.count()`, `func.max()`

In [14]:
# Variable st represents students table aliased as 'a'
st = students.alias('a')
print('Original table name: ', students.name)
print('Alias table name: ', st.name)
print('')

# Run select all against aliased table
s = st.select().where(st.c.id == 1)
result = conn.execute(s)
result.fetchall()

Original table name:  students
Alias table name:  a

2021-06-10 16:40:36,779 INFO sqlalchemy.engine.Engine SELECT a.id, a.name, a.lastname 
FROM students AS a 
WHERE a.id = ?
2021-06-10 16:40:36,781 INFO sqlalchemy.engine.Engine [generated in 0.00198s] (1,)


[(1, 'Ravi', 'Kapoor')]

In [15]:
# Select students in alphabetical order
s = select(students).order_by(
    asc(students.c.name)
)
print(str(s))

SELECT students.id, students.name, students.lastname 
FROM students ORDER BY students.name ASC


In [16]:
# Count rows in table
s = select(func.count(students.c.id))
print(str(s))

SELECT count(students.id) AS count_1 
FROM students


### Working with Multiple Tables

In [17]:
# Define new table
addresses = Table(
   'addresses', meta, 
   Column('id', Integer, primary_key = True), 
   Column('st_id', Integer, ForeignKey('students.id')), # Foreign key to students table
   Column('postal_add', String), 
   Column('email_add', String)
)

# Create the table
meta.create_all(engine)

# Populate new table
conn.execute(addresses.insert(), [
   {'st_id':1, 'postal_add':'Shivajinagar Pune', 'email_add':'ravi@gmail.com'},
   {'st_id':1, 'postal_add':'ChurchGate Mumbai', 'email_add':'kapoor@gmail.com'},
   {'st_id':3, 'postal_add':'Jubilee Hills Hyderabad', 'email_add':'komal@gmail.com'},
   {'st_id':5, 'postal_add':'MG Road Bangaluru', 'email_add':'as@yahoo.com'},
   {'st_id':2, 'postal_add':'Cannought Place new Delhi', 'email_add':'admin@khanna.com'},
]);

2021-06-10 16:40:36,848 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-10 16:40:36,849 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2021-06-10 16:40:36,851 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-10 16:40:36,854 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2021-06-10 16:40:36,855 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-10 16:40:36,856 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("addresses")
2021-06-10 16:40:36,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-10 16:40:36,859 INFO sqlalchemy.engine.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	st_id INTEGER, 
	postal_add VARCHAR, 
	email_add VARCHAR, 
	PRIMARY KEY (id), 
	FOREIGN KEY(st_id) REFERENCES students (id)
)


2021-06-10 16:40:36,862 INFO sqlalchemy.engine.Engine [no key 0.00273s] ()
2021-06-10 16:40:36,865 INFO sqlalchemy.engine.Engine COMMIT
2021-06-10 16:40:36,867 INFO sqlalchemy.engine.Engine INSERT INTO addresses (st_id, postal_

**Implicit Joins:** Simple implicit join by passing both tables as arguments of `select` and defining join condition in `where` clause

In [18]:
# Select all from both joining on foreign key
s = select(
    students,
    addresses
).where(
    students.c.id == addresses.c.st_id
)
print(str(s))

SELECT students.id, students.name, students.lastname, addresses.id AS id_1, addresses.st_id, addresses.postal_add, addresses.email_add 
FROM students, addresses 
WHERE students.id = addresses.st_id


**Join:** Method on tables
- Use `select_from()` method on `select` to explicitly set left hand side of join
- Can be multiple ways of phrasing
- Just specifying table name in `select` is equivalent to all columns from that table

In [19]:
# Join on its own
j = students.join(
    addresses,
    students.c.id == addresses.c.st_id,
    isouter = True # outer join - default is false
)
print(str(j))

students LEFT OUTER JOIN addresses ON students.id = addresses.st_id


In [20]:
# As part of select statement
s = select(
    students.c.name,
    students.c.lastname,
    addresses
).select_from(
    j # equivalent to block commented code below
    # students.join(
    #     addresses,
    #     students.c.id == addresses.c.st_id,
    #     isouter=True
    # )
)
print(str(s))

SELECT students.name, students.lastname, addresses.id, addresses.st_id, addresses.postal_add, addresses.email_add 
FROM students LEFT OUTER JOIN addresses ON students.id = addresses.st_id


In [21]:
# An equivalent formulation
s = select(
    students.c.name,
    students.c.lastname,
    addresses
).select_from(
    students
).join(
    addresses,
    students.c.id == addresses.c.st_id,
    isouter=True
)
print(str(s))

SELECT students.name, students.lastname, addresses.id, addresses.st_id, addresses.postal_add, addresses.email_add 
FROM students LEFT OUTER JOIN addresses ON students.id = addresses.st_id


## SQLAlchemy ORM

**SQLAlchemy ORM:** Main objective it to associate user-defined Python classes with databases and objects with rows
- Is constructed on top of SQL Expression Language
- High-level and abstracted usage of the Expression Language

### Declaring Mappings

**Declarative System:** Classes created include directives to describe actual database table they are mapped to.

**Declarative Base:** Class that stores catalogue of classes and mapped tables in Declarative system
- Usually only one instance
- Has a `MetaData` object as an attribute

**Mapped Classes:** Subclasses of declarative base that will be mapped to tables
- Must include `__tablename__` attribute giving name of corresponding table
- Includes `Column`s with their datatypes

**Session:** Handle to the database - it encapsulates a single connection to the DB
- `Session` class defined by the engine and a session object is an instantiation of this

In [22]:
# Create engine that describes database
engine = create_engine(
    os.path.join('sqlite:///', SALES_DB_PATH),
    echo = True
)
Base = declarative_base()

class Customers(Base):
   __tablename__ = 'customers'
   id = Column(Integer, primary_key=True)

   name = Column(String)
   address = Column(String)
   email = Column(String)

# Uses metadata of declarative base to turn engine into source of connection
# Issues create table statements for tables that don't yet exist
Base.metadata.create_all(engine)

2021-06-10 16:40:36,979 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-10 16:40:36,981 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customers")
2021-06-10 16:40:36,983 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-10 16:40:36,985 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("customers")
2021-06-10 16:40:36,987 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-10 16:40:36,989 INFO sqlalchemy.engine.Engine 
CREATE TABLE customers (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	address VARCHAR, 
	email VARCHAR, 
	PRIMARY KEY (id)
)


2021-06-10 16:40:36,991 INFO sqlalchemy.engine.Engine [no key 0.00182s] ()
2021-06-10 16:40:37,000 INFO sqlalchemy.engine.Engine COMMIT


Now create a session

In [23]:
Session = sessionmaker(bind=engine)
session = Session()

**Adding Objects:** Equivalent to inserting rows
- Instantiate an instance of mapped class and call `add()` method on session
- Transactions are pending until flushed using `commit()` method on session

In [24]:
# Add row to customers table
c1 = Customers(
    name = 'Ravi Kumar',
    address = 'Station Road Nanded',
    email = 'ravi@gmail.com'
)
session.add(c1)
session.commit()

2021-06-10 16:40:37,042 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-10 16:40:37,044 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2021-06-10 16:40:37,045 INFO sqlalchemy.engine.Engine [generated in 0.00112s] ('Ravi Kumar', 'Station Road Nanded', 'ravi@gmail.com')
2021-06-10 16:40:37,050 INFO sqlalchemy.engine.Engine COMMIT


In [25]:
# Can add multiple at once by passing list
session.add_all([
   Customers(name = 'Komal Pande', address = 'Koti, Hyderabad', email = 'komal@gmail.com'), 
   Customers(name = 'Rajender Nath', address = 'Sector 40, Gurgaon', email = 'nath@gmail.com'), 
   Customers(name = 'S.M.Krishna', address = 'Budhwar Peth, Pune', email = 'smk@gmail.com')]
)
session.commit()

2021-06-10 16:40:37,073 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-10 16:40:37,074 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2021-06-10 16:40:37,075 INFO sqlalchemy.engine.Engine [cached since 0.03123s ago] ('Komal Pande', 'Koti, Hyderabad', 'komal@gmail.com')
2021-06-10 16:40:37,078 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2021-06-10 16:40:37,079 INFO sqlalchemy.engine.Engine [cached since 0.03515s ago] ('Rajender Nath', 'Sector 40, Gurgaon', 'nath@gmail.com')
2021-06-10 16:40:37,081 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2021-06-10 16:40:37,081 INFO sqlalchemy.engine.Engine [cached since 0.03746s ago] ('S.M.Krishna', 'Budhwar Peth, Pune', 'smk@gmail.com')
2021-06-10 16:40:37,083 INFO sqlalchemy.engine.Engine COMMIT


### Basic Select Statements

**Query:** SELECT statements are generated by query objects
- Query objects can be generated by `query(<mappedClass>)` method on Session
- Or by instantiating `Query` class directly through `Query(<mappedClass>, <session>)`
- Limit columns by passing specific column attributes as argument of `query()`
- Rename columns using `label()` method on columns
- Can call `str(<query>)` to get equivalent SQL statement
- Convert to equivalent Core Select object via `statement` attribute

**Accessing Results:**
- *Directly:* Query objects are iterable with items corresponding to rows
    - If whole MappedClass selected in query then items are mappedClass instances
    - If individual columns selected then items are of column dtype (can still be accessed like a dict with column name)
    - I think the query is only run against the database when iteration starts?
- *`all`:* Method on query object that runs the query and returns list of results
    - Valuable because it gives snapshot of query run that is stored - accessing directly from query object multiple times is inefficient and results may change
- *Get:* Quickly retrieve row from Query object by primary key using `get(<primary_key>)` method

In [26]:
# Select all
q = session.query(Customers)
print('Equivalent SQL')
print('--------------')
print(str(q))
print('')

# Print results
for row in q:
    print('id: ', row.id, ', name: ', row.name, ', address: ', row.address, ', email: ', row.email)

Equivalent SQL
--------------
SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers

2021-06-10 16:40:37,111 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-10 16:40:37,113 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers
2021-06-10 16:40:37,114 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ()
id:  1 , name:  Ravi Kumar , address:  Station Road Nanded , email:  ravi@gmail.com
id:  2 , name:  Komal Pande , address:  Koti, Hyderabad , email:  komal@gmail.com
id:  3 , name:  Rajender Nath , address:  Sector 40, Gurgaon , email:  nath@gmail.com
id:  4 , name:  S.M.Krishna , address:  Budhwar Peth, Pune , email:  smk@gmail.com


In [27]:
# Limit columns
q = session.query(
    Customers.id,
    Customers.name.label('full_name')
)
print('Equivalent SQL')
print('--------------')
print(str(q))
print('')

result = q.first()
print('\nFirst result: ', result)
print('Data types: ', (type(result.id), type(result.full_name)))

Equivalent SQL
--------------
SELECT customers.id AS customers_id, customers.name AS full_name 
FROM customers

2021-06-10 16:40:37,148 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS full_name 
FROM customers
 LIMIT ? OFFSET ?
2021-06-10 16:40:37,151 INFO sqlalchemy.engine.Engine [generated in 0.00497s] (1, 0)

First result:  (1, 'Ravi Kumar')
Data types:  (<class 'int'>, <class 'str'>)


In [28]:
# Run query and store results in list
q = session.query(Customers)
print('Query is run now:')
result = q.all()
print('\nResults are printed now:')

# Print results
for row in result:
    print('id: ', row.id, ', name: ', row.name, ', address: ', row.address, ', email: ', row.email)

Query is run now:
2021-06-10 16:40:37,171 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers
2021-06-10 16:40:37,175 INFO sqlalchemy.engine.Engine [cached since 0.06262s ago] ()

Results are printed now:
id:  1 , name:  Ravi Kumar , address:  Station Road Nanded , email:  ravi@gmail.com
id:  2 , name:  Komal Pande , address:  Koti, Hyderabad , email:  komal@gmail.com
id:  3 , name:  Rajender Nath , address:  Sector 40, Gurgaon , email:  nath@gmail.com
id:  4 , name:  S.M.Krishna , address:  Budhwar Peth, Pune , email:  smk@gmail.com


In [29]:
# Alternative syntax
q = Query(Customers, session)
print(str(q))
print('')

result = q.all()

SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers

2021-06-10 16:40:37,200 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers
2021-06-10 16:40:37,204 INFO sqlalchemy.engine.Engine [cached since 0.0913s ago] ()


In [30]:
# Equivalent Select object
q = session.query(Customers)
type(q.statement)

sqlalchemy.sql.selectable.Select

In [31]:
# Customer with id = 2
q = session.query(Customers)
result = q.get(2)
print('Name: ', result.name)

Name:  Komal Pande


### WHERE Clauses

**Filters:** WHERE clauses are applying using `filter(<cond>)` method on query objects
- Columns are referenced using corresponding attribute of class
- `str(<query>)` doesn't show specific values - access by converting to Core Select object and using `.compile().params`

**Filter Operators:** Apply to columns in `filter`
- Ex: `.like()`, `.in_(<list>)`, `or_(<cond1>, <cond2>)`, `and_(<cond1>, <cond2>)` (the `and_` is actually unnecessary, can just separate conditions with commas in `filter()`)

In [32]:
# Rows with id > 2
q = session.query(Customers).filter(Customers.id > 2)
print('Equivalent SQL')
print('--------------')
print(str(q))
print('')
print('Params: ', q.statement.compile().params)

Equivalent SQL
--------------
SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.id > ?

Params:  {'id_1': 2}


In [33]:
# Query with AND condition
q = session.query(Customers).filter(
    Customers.id>2,
    Customers.name.like('Ra%')
)
print(str(q))

SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.id > ? AND customers.name LIKE ?


In [34]:
# Equivalent formulation
q = session.query(Customers).filter(and_(
    Customers.id>2,
    Customers.name.like('Ra%')
))
print(str(q))

SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.id > ? AND customers.name LIKE ?


### Textual SQL

**TextClauses in ORM:** Apply `TextClause`s from Core to Query objects flexibly
- Can apply using `from_statement` method on Query object
- `filter` can automatically translate into WHERE clause - this way specific values are stores in `str(<query>)`
- Bind parameters using colon syntax and `params()` method on Query object

In [35]:
# Select all and display first result
stmt = text('SELECT * FROM customers')
q = session.query(Customers).from_statement(stmt)
print('Query SQL: ', str(q))
print('')

result = q.first()
print('')
print('First result name: ', result.name)

Query SQL:  SELECT * FROM customers

2021-06-10 16:40:37,324 INFO sqlalchemy.engine.Engine SELECT * FROM customers
2021-06-10 16:40:37,332 INFO sqlalchemy.engine.Engine [generated in 0.00853s] ()

First result name:  Ravi Kumar


In [36]:
# Filter with textual SQL
q = session.query(Customers).filter(text('id == 4'))
print('Query SQL')
print('---------')
print(str(q))
print('')

result = q.one() # Throws an error if more than one result
print('')
print('Unique result name: ', result.name)

Query SQL
---------
SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE id == 4

2021-06-10 16:40:37,385 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE id == 4
2021-06-10 16:40:37,388 INFO sqlalchemy.engine.Engine [generated in 0.00253s] ()

Unique result name:  S.M.Krishna


In [37]:
# Filter with textual SQL and parameter binding
q = session.query(Customers).filter(text('id == :id')).params(id = 4)
print('Query SQL')
print('---------')
print(str(q))
print('')

# Print params
print('Bound params: ', q.statement.compile().params)
print('')

result = q.one() # Throws an error if more than one result
print('')
print('Unique result name: ', result.name)

Query SQL
---------
SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE id == ?

Bound params:  {'id': 4}

2021-06-10 16:40:37,410 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE id == ?
2021-06-10 16:40:37,413 INFO sqlalchemy.engine.Engine [generated in 0.00245s] (4,)

Unique result name:  S.M.Krishna


### Working with Multiple Tables



In [38]:
# Define new MappedClass
class Invoices(Base):
   __tablename__ = 'invoices'
   
   id = Column(Integer, primary_key = True)
   custid = Column(Integer, ForeignKey('customers.id'))
   invno = Column(Integer)
   amount = Column(Integer)
   customer = relationship("Customers", back_populates = "invoices")


# Add relationship to Customers MappedClass
Customers.invoices = relationship(
    "Invoices",
    order_by = Invoices.id,
    back_populates = "customer"
)

# Create tables that don't exist yet
Base.metadata.create_all(engine)

2021-06-10 16:40:37,439 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-10 16:40:37,440 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customers")
2021-06-10 16:40:37,441 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-10 16:40:37,443 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("invoices")
2021-06-10 16:40:37,445 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-10 16:40:37,447 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("invoices")
2021-06-10 16:40:37,448 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-10 16:40:37,450 INFO sqlalchemy.engine.Engine 
CREATE TABLE invoices (
	id INTEGER NOT NULL, 
	custid INTEGER, 
	invno INTEGER, 
	amount INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(custid) REFERENCES customers (id)
)


2021-06-10 16:40:37,451 INFO sqlalchemy.engine.Engine [no key 0.00080s] ()
2021-06-10 16:40:37,455 INFO sqlalchemy.engine.Engine COMMIT


In [39]:
# Create new customer
c1 = Customers(
    name = "Gopal Krishna",
    address = "Bank Street Hydarebad",
    email = "gk@gmail.com"
)

# Can create invoices for customer by passing list as invoices attribute
c1.invoices = [
    Invoices(invno = 10, amount = 15000),
    Invoices(invno = 14, amount = 3850)
]

# Adding customer automatically adds invoices as well
session.add(c1)
session.commit()

2021-06-10 16:40:37,480 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2021-06-10 16:40:37,483 INFO sqlalchemy.engine.Engine [generated in 0.00351s] ('Gopal Krishna', 'Bank Street Hydarebad', 'gk@gmail.com')
2021-06-10 16:40:37,492 INFO sqlalchemy.engine.Engine INSERT INTO invoices (custid, invno, amount) VALUES (?, ?, ?)
2021-06-10 16:40:37,497 INFO sqlalchemy.engine.Engine [generated in 0.00517s] (5, 10, 15000)
2021-06-10 16:40:37,505 INFO sqlalchemy.engine.Engine INSERT INTO invoices (custid, invno, amount) VALUES (?, ?, ?)
2021-06-10 16:40:37,506 INFO sqlalchemy.engine.Engine [cached since 0.01354s ago] (5, 14, 3850)
2021-06-10 16:40:37,511 INFO sqlalchemy.engine.Engine COMMIT


In [40]:
# Select all rows from Invoices table
q = session.query(Invoices)
result = q.all()
print('\nInvoices:')

for row in result:
    print('id :', row.id, ', cust_id:', row.custid, ', amount: ', row.amount)

2021-06-10 16:40:37,540 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-10 16:40:37,544 INFO sqlalchemy.engine.Engine SELECT invoices.id AS invoices_id, invoices.custid AS invoices_custid, invoices.invno AS invoices_invno, invoices.amount AS invoices_amount 
FROM invoices
2021-06-10 16:40:37,546 INFO sqlalchemy.engine.Engine [generated in 0.00242s] ()

Invoices:
id : 1 , cust_id: 5 , amount:  15000
id : 2 , cust_id: 5 , amount:  3850


In [41]:
# Add more rows
rows = [
   Customers(
      name = "Govind Pant", 
      address = "Gulmandi Aurangabad",
      email = "gpant@gmail.com",
      invoices = [
          Invoices(invno = 3, amount = 10000), 
          Invoices(invno = 4, amount = 5000)
      ]
   ),

   Customers(
      name = "Govind Kala", 
      address = "Gulmandi Aurangabad", 
      email = "kala@gmail.com", 
      invoices = [
          Invoices(invno = 7, amount = 12000),
          Invoices(invno = 8, amount = 18500)
      ]
   ),

   Customers(
      name = "Abdul Rahman", 
      address = "Rohtak", 
      email = "abdulr@gmail.com",
      invoices = [
          Invoices(invno = 9, amount = 15000), 
          Invoices(invno = 11, amount = 6000)
       ]
   )
]

session.add_all(rows)
session.commit()

2021-06-10 16:40:37,569 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2021-06-10 16:40:37,571 INFO sqlalchemy.engine.Engine [cached since 0.09079s ago] ('Govind Pant', 'Gulmandi Aurangabad', 'gpant@gmail.com')
2021-06-10 16:40:37,572 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2021-06-10 16:40:37,573 INFO sqlalchemy.engine.Engine [cached since 0.09323s ago] ('Govind Kala', 'Gulmandi Aurangabad', 'kala@gmail.com')
2021-06-10 16:40:37,575 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, address, email) VALUES (?, ?, ?)
2021-06-10 16:40:37,575 INFO sqlalchemy.engine.Engine [cached since 0.0957s ago] ('Abdul Rahman', 'Rohtak', 'abdulr@gmail.com')
2021-06-10 16:40:37,580 INFO sqlalchemy.engine.Engine INSERT INTO invoices (custid, invno, amount) VALUES (?, ?, ?)
2021-06-10 16:40:37,584 INFO sqlalchemy.engine.Engine [cached since 0.09142s ago] (6, 3, 10000)
2021-06-10 16:40:37,586 INFO sqla

**Implicit Join:** Pass both mapped classes to `query()` and specify join condition in `filter()`
- If full classes are passed then items in `Query` iterable are instances of classes
- If certain columns passed then items are objects of column datatype

In [42]:
# Select * on implicit join
q = session.query(Customers, Invoices).filter(Customers.id == Invoices.custid)
print('Equivalent SQL')
print('--------------')
print(str(q))
print('')

results = q.all()
print('')
print('First result: ', results[0])

print('\nIterating over all results:')
for c, i in results:
    print('ID: {} Name: {} Invoice No: {} Amount: {}'.format(c.id,c.name, i.invno, i.amount))

Equivalent SQL
--------------
SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email, invoices.id AS invoices_id, invoices.custid AS invoices_custid, invoices.invno AS invoices_invno, invoices.amount AS invoices_amount 
FROM customers, invoices 
WHERE customers.id = invoices.custid

2021-06-10 16:40:37,635 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-10 16:40:37,638 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email, invoices.id AS invoices_id, invoices.custid AS invoices_custid, invoices.invno AS invoices_invno, invoices.amount AS invoices_amount 
FROM customers, invoices 
WHERE customers.id = invoices.custid
2021-06-10 16:40:37,640 INFO sqlalchemy.engine.Engine [generated in 0.00244s] ()

First result:  (<__main__.Customers object at 0x111c12940>, <__main__.Invo

In [43]:
# Select certain columns on implicit join
q = session.query(
    Customers.id,
    Customers.name,
    Invoices.invno,
    Invoices.amount
).filter(Customers.id == Invoices.custid)
print('Equivalent SQL')
print('--------------')
print(str(q))
print('')

results = q.all()
print('')
print('First result: ', results[0])

print('\nIterating over all results:')
for row in results:
    print('ID: {} Name: {} Invoice No: {} Amount: {}'.format(row.id, row.name, row.invno, row.amount))

Equivalent SQL
--------------
SELECT customers.id AS customers_id, customers.name AS customers_name, invoices.invno AS invoices_invno, invoices.amount AS invoices_amount 
FROM customers, invoices 
WHERE customers.id = invoices.custid

2021-06-10 16:40:37,667 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, invoices.invno AS invoices_invno, invoices.amount AS invoices_amount 
FROM customers, invoices 
WHERE customers.id = invoices.custid
2021-06-10 16:40:37,668 INFO sqlalchemy.engine.Engine [generated in 0.00118s] ()

First result:  (5, 'Gopal Krishna', 10, 15000)

Iterating over all results:
ID: 5 Name: Gopal Krishna Invoice No: 10 Amount: 15000
ID: 5 Name: Gopal Krishna Invoice No: 14 Amount: 3850
ID: 6 Name: Govind Pant Invoice No: 3 Amount: 10000
ID: 6 Name: Govind Pant Invoice No: 4 Amount: 5000
ID: 7 Name: Govind Kala Invoice No: 7 Amount: 12000
ID: 7 Name: Govind Kala Invoice No: 8 Amount: 18500
ID: 8 Name: Abdul Rahman Invoice 

**Join:** `join` method on Query object syntax `join(<mappedClass>, <cond1>)`
- `outerjoin` method for left outer join
- Condition can be dropped for join on foreign keys
- Can join with Table objects from Core (see subquery below)
- Use `select_from()` method on query to explicitly set left side of join

In [44]:
# Just select customers.
q = session.query(Customers).join(
    Invoices,
    Customers.id == Invoices.custid
)
print(str(q))

SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers JOIN invoices ON customers.id = invoices.custid


In [45]:
# Select all
q = session.query(
    Customers,
    Invoices
).join(
    Invoices,
    Customers.id == Invoices.custid
)
print(str(q))

SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email, invoices.id AS invoices_id, invoices.custid AS invoices_custid, invoices.invno AS invoices_invno, invoices.amount AS invoices_amount 
FROM customers JOIN invoices ON customers.id = invoices.custid


In [46]:
# Explicitly set left side of join
q = session.query(
    Customers.name,
    Invoices.amount
).select_from(
    Customers
).outerjoin(
    Invoices,
    Customers.id == Invoices.custid
)
print(str(q))

SELECT customers.name AS customers_name, invoices.amount AS invoices_amount 
FROM customers LEFT OUTER JOIN invoices ON customers.id = invoices.custid


**Subqueries:** Created by `subquery()` method on Query object, acts like a Table object from Core
- Useful to combine with `alias()`

In [47]:
# Create subquery object - acts like a table
sub = session.query(
   Invoices.custid, func.count('*').label('invoice_count')
).group_by(Invoices.custid).subquery().alias('i')
print(str(sub))
print('\nTable alias: ', sub.name)

SELECT invoices.custid, count(:count_1) AS invoice_count 
FROM invoices GROUP BY invoices.custid

Table alias:  i


In [48]:
# Use subquery in outer query
q = session.query(
    Customers,
    sub.c.invoice_count # Use syntax for Tables
).outerjoin(
    sub,
    Customers.id == sub.c.custid
).order_by(Customers.id)
print(str(q))

SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email, i.invoice_count AS i_invoice_count 
FROM customers LEFT OUTER JOIN (SELECT invoices.custid AS custid, count(?) AS invoice_count 
FROM invoices GROUP BY invoices.custid) AS i ON customers.id = i.custid ORDER BY customers.id
